# MVD 4. cvičení

## 1. část - Načtení dat

Po rozbalení archive.zip uvidíte articles csv soubor. Tento soubor pochází z [Kaggle datasetů](https://www.kaggle.com/hsankesara/medium-articles) a obsahuje malé množství Medium článků k tématům ML, AI a data science. K úloze dnešního cvičení bude stačit využítí dat s názvy a obsahy článků (title a text).


### Příprava dat

Pro přípravu dat se použivá různá sekvence kroků. Je doporučeno na následující kroky vytvořit samostatnou funkci, aby bylo možné zpracovat i vyhledávaný výraz při testování. Dnešní cvičení by mělo obsahovat následující kroky:

1. Převést všechen text na lower case
2. Odstranění interpunkce a všech speciálních znaků (apostrof, ...)
3. Aplikace lemmatizátoru

Pozn.: Jedná se pouze o jednoduchý preprocessing, v praxi je často potřeba použití více kroků. Tato aplikace by měla například problém s čísly (desetinná čísla, čísla vyhledávaná slovně). 

Pro lemmatizaci použijte knihovnu spaCy.

In [4]:
# Instalace spaCy z Jupyter Notebooku
import sys
!{sys.executable} -m pip install spacy

# Stažení modelu pro angličtinu
!{sys.executable} -m spacy download en

You should consider upgrading via the 'C:\Users\kanyk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


You should consider upgrading via the 'C:\Users\kanyk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [5]:
def preprocess(lists):
    head_names = lists.columns.values
    for i in range(0, len(head_names)):
        for j in range(0, len(lists[head_names[i]])):
            lists[head_names[i]][j] = lists[head_names[i]][j].lower()
            lists[head_names[i]][j] = re.sub(r'[^A-Za-z 0-9]', '', lists[head_names[i]][j])
            lemm_words = " ".join([token.lemma_ for token in lemmatizer(lists[head_names[i]][j]) if " " not in str(token)])
            lists[head_names[i]][j] = lemm_words
    return lists

In [6]:
import spacy
lemmatizer = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
# Lemmatizace textu př.:
# " ".join([token.lemma_ for token in lemmatizer(text)])

import pandas as pd
import re 
data = pd.read_csv('archive.zip')
articles = [list(data['title']), list(data['text'])]
articles = preprocess(data[['title', 'text']])


C:\Users\kanyk\AppData\Local\Temp/ipykernel_11476/3376337523.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lists[head_names[i]][j] = lists[head_names[i]][j].lower()
C:\Users\kanyk\AppData\Local\Temp/ipykernel_11476/3376337523.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lists[head_names[i]][j] = re.sub(r'[^A-Za-z 0-9]', '', lists[head_names[i]][j])
C:\Users\kanyk\AppData\Local\Temp/ipykernel_11476/3376337523.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

## 2. část - Vytvoření invertovaného indexu

Před další prací s textem je potřeba vytvořit invertovaný index, který poté usnadní práci. Invertovaný index bude slovník, kde klíčem bude slovo a hodnotou bude list s id dokumentů (index), které dané slovo obsahují.

Pozn.: Je potřeba vytvořit dva invertované indexy - jeden pro title a druhý pro text.

In [7]:
def make_invert_index(text_list):
    slovnik = {w : [0] for w in set(text_list[0].split(" "))}
    for i in range(1, len(text_list)):
        words_set = list(set(text_list[i].split(" ")))
        {slovnik[words_set[j]].append(i) if words_set[j] in slovnik.keys() else slovnik.update({words_set[j]: [i]}) for j in range(0, len(words_set))}
    return slovnik
inv_index_titles = make_invert_index(articles.title)
inv_index_texts = make_invert_index(articles.text)

## 3. část - Implementace TF-IDF

Připravení funkce pro výpočet TF-IDF po příchodu dotazu. Funkce *tf_idf* by měla pracovat s dotazem, jedním invertovaným indexem a s danými dokumenty. Vrátit by měla list obsahující skóre pro každý dokument.

<br>
<center>
$
score(q,d) = TF\_IDF(q,d) = \sum\limits_{w \in q \cap d} c(w, q) c(w, d) log(\frac{M+1}{df(w)})
$
</center>

$q$ ... dotaz<br>
$d$ ... dokument<br>
$c(w, q)$ ... kolikrát je slovo *w* v dotazu *q*<br>
$M$ ... celkový počet dokumentů<br>
$df(w)$ ... počet dokumentů, ve kterých se nachází slovo *w*

In [71]:
import math
def find_count_occur(text):
    c_w = {}
    for w in text:
        if w in c_w.keys():
            c_w[w] = c_w[w] + 1 
        else:
            c_w.update({w : 1})
    return c_w

def tf_idf(query, inv_index, doc):
    occur_query = find_count_occur(query)
    score = []
    suma = 0
    for w in query:
        coun_occur = find_count_occur(doc)
        if w not in coun_occur.keys():
            c_w_d = 0
        else:
            c_w_d = coun_occur[w]
        suma += occur_query[w]*c_w_d*math.log((1+1)/len(inv_index[w]))
    score.append(suma)
    return score 

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer
query = "2016 machine intelligence landscape"
vect = TfidfVectorizer()
vect.fit_transform(articles.text[1].split(" "))
words = vect.get_feature_names_out()
print(tf_idf(list(words), inv_index_texts, articles.text[3]))


[0.0]


## 4. část - Použití a testování TF-IDF

Nyní lze získat skóre pro titulky nebo text. Následujícím krokem je sjednocení výsledného skóre pro ohodnocení celého dokumentu. V případě dvou hodnot si vystačíme s parametrem $\alpha$, který nám určuje jakou váhu má titulek a jakou samotný text dokumentu. <br>

<center>
$
score(q,d) = \alpha \; TF\_IDF\_title(q,d) + (1-\alpha) \; TF\_IDF\_text(q,d)
$
</center>

Při nastavení parametru $\alpha$ na hodnotu 0.7 a vyhledávání dotazu "coursera vs udacity machine learning" by výsledky měly vypadat následovně:

![output](sample_output.png)

In [6]:
alpha = 0.7
query = "coursera vs udacity machine learning"
score_titles = [x * alpha for x in tf_idf(query, inv_index_titles, articles.title)]
score_articles = [x * (1-alpha) for x in tf_idf(query, inv_index_texts, articles.text)]
score = [score_titles[i]+score_articles[i] for i in range(0, len(score_titles))]
data = articles.assign(score=score)
data.sort_values(by='score', ascending=False).head(10)

,title,text,score
276,coursera vs udacity for machine learn hacker noon,2018 be an exciting time for student of machin...,25.567283
143,every single machine learn course on the inter...,a year and a half ago I drop out of one of the...,18.018200
99,every single machine learn course on the inter...,a year and a half ago I drop out of one of the...,18.018200
19,every single machine learn course on the inter...,a year and a half ago I drop out of one of the...,18.018200
67,every single machine learn course on the inter...,a year and a half ago I drop out of one of the...,18.018200
209,in defense of skepticism about deep learn gary...,in a recent appraisal of deep learning marcus ...,9.013928
327,in defense of skepticism about deep learn gary...,in a recent appraisal of deep learning marcus ...,9.013928
13,machine learning be fun adam geitgey medium,update this article be part of a series check ...,7.159569
36,machine learning be fun adam geitgey medium,update this article be part of a series check ...,7.159569
37,the current state of machine intelligence shiv...,the 2016 machine intelligence landscape and po...,6.790985
